## Libraries

In [1]:
import numpy as np
import pandas as pd
import xlrd
import os

In [2]:
import sys
np.set_printoptions(threshold=np.inf)

## Mount Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#os.listdir("/content/drive/My Drive/Colab Notebooks")

In [ ]:
os.listdir(".")

In [ ]:
housePath = "/content/drive/My Drive/Colab Notebooks/house_prices.xlsx"

# Raw Population to dataframe

In [3]:
# xl3 = pd.ExcelFile("/content/drive/My Drive/Colab Notebooks/idaho_county_population_2000_to_2010.xlsx")
# xl4 = pd.ExcelFile("/content/drive/My Drive/Colab Notebooks/idaho_county_population_2010_to_2018.xlsx")
xl3 = pd.ExcelFile("./Data/idaho_county_population_2000_to_2010.xlsx")
xl4 = pd.ExcelFile("./Data/idaho_county_population_2010_to_2018.xlsx")

In [4]:
dfpop1 = xl3.parse("main")
dfpop1.to_pickle('rawPop1.pkl')
dfpop2 = xl4.parse("main")
dfpop2.to_pickle('rawPop2.pkl')

### This code to clean and prep population 00-10 from pickle

In [5]:
rawPop1 = pd.read_pickle('rawPop1.pkl')
temp1 = rawPop1.copy()
temp1 = temp1.iloc[2:48]
temp1 = temp1.replace(['County','\.','\s'],'', regex = True)
cTemp = temp1['table with row headers in column A and column headers in rows 3 through 4 (leading dots indicate sub-parts)']
temp1 = temp1.drop(['table with row headers in column A and column headers in rows 3 through 4 (leading dots indicate sub-parts)',
                    'Unnamed: 1','Unnamed: 12','Unnamed: 13'], axis = 1)
temp1 = temp1.astype(int)
temp1.columns = temp1.iloc[0]
temp1 = temp1.iloc[2:]
temp1['County'] = cTemp
pop1 = temp1.copy()
pop1.head(3)


2,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,County
4,303328,313896,321616,327393,334926,348755,363498,375368,382618,388577,Ada
5,3477,3495,3559,3624,3693,3817,3788,3949,4021,4000,Adams
6,75728,76296,76487,76312,76834,77419,78491,79338,80609,81994,Bannock


### This code to clean and prep population 10-18 from pickle

In [6]:
rawPop2 = pd.read_pickle('rawPop2.pkl')
temp2 = rawPop2.copy()
temp2 = temp2.replace(['Population Estimate \(as of July 1\) \- ','County, Idaho'],'', regex = True)
temp2.columns = temp2.iloc[0]
#temp2 = temp_df.iloc[2:]
cT2 = temp2['Geography']
temp2 = temp2.drop(['Id','Id2','Geography','April 1, 2010 - Census',
                     'April 1, 2010 - Estimates Base'], axis = 1)
temp2 = temp2.astype(int)
temp2['County'] = cT2
temp2 = temp2.iloc[1:]
temp2 = temp2.replace('\s','', regex = True)

pop2 = temp2.copy()
pop2.head(3)

,2010,2011,2012,2013,2014,2015,2016,2017,2018,County
1,393350,401235,408930,416080,425540,433047,444824,456885,469966,Ada
2,3962,3980,3907,3862,3882,3889,3940,4132,4250,Adams
3,83019,83628,83744,83449,83534,84216,84794,85641,87138,Bannock


### Merge the cleaned data frames for the 00-09 pop and 10-18 pop and pickle the result (totalPop)

In [8]:
totalPop = pd.merge(pop2,pop1, on = 'County')

# Reorder columns, not all column titles are str some are int, going to 
# transpose df so will turn them all into factors then


totalPop = totalPop[["County",2000,2001,2002,2003,2004,2005,2006,2007,
                    2008,2009,"2010","2011","2012","2013","2014","2015",
                    "2016","2017","2018"]]


totalPop.to_pickle('totalPop.pkl')
totalPop.head(5)

,County,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Ada,303328,313896,321616,327393,334926,348755,363498,375368,382618,388577,393350,401235,408930,416080,425540,433047,444824,456885,469966
1,Adams,3477,3495,3559,3624,3693,3817,3788,3949,4021,4000,3962,3980,3907,3862,3882,3889,3940,4132,4250
2,Bannock,75728,76296,76487,76312,76834,77419,78491,79338,80609,81994,83019,83628,83744,83449,83534,84216,84794,85641,87138
3,BearLake,6424,6394,6219,6219,6170,6077,6071,6049,6027,6014,5969,5955,5892,5948,5926,5879,5926,6031,6050
4,Benewah,9186,8968,8917,8901,8907,9053,9260,9296,9385,9286,9290,9172,9125,9028,9032,8990,9015,9165,9226


### From totalPop.pkl to metroPop, then pickle

In [9]:
totalPop = pd.read_pickle('totalPop.pkl')
totalPop = totalPop.iloc[[0,7,13,22,36],:]

temp3 = totalPop.copy()
temp3 = temp3.T
temp3.columns = temp3.iloc[0]
temp3 = temp3.iloc[1:]
temp3 = temp3.reset_index()
temp3 = temp3.rename(columns={'index':'Year'})

# Add the columns together for a total population
temp3['TotalPop'] = temp3['Ada']+temp3['Boise']+temp3['Canyon']+temp3['Gem']+temp3['Owyhee']
metroPop = temp3.copy()
pop_year_totals = metroPop.drop(columns=['Ada','Boise','Canyon','Gem','Owyhee'])

metroPop = metroPop.astype('int')

metroPop['Year'] = metroPop['Year'].astype(str)
metroPop['Year'] = metroPop['Year'].astype('datetime64[Y]')

metroPop.to_pickle('metroPop.pkl')
metroPop.head(5)

County,Year,Ada,Boise,Canyon,Gem,Owyhee,TotalPop
0,2000-01-01,303328,6702,133082,15215,10690,469017
1,2001-01-01,313896,6733,139179,15393,10877,486078
2,2002-01-01,321616,6854,145160,15488,10876,499994
3,2003-01-01,327393,6977,151395,15693,11033,512491
4,2004-01-01,334926,7004,157130,15925,10990,525975


# House Count

In [ ]:
## Load House excel files and pickle

xl5 = pd.ExcelFile("./Data/idaho_housing_unit_count_2000_to_2010.xlsx")
xl6 = pd.ExcelFile("./Data/idaho_housing_unit_count_2010_to_2018.xlsx")

dfH1 = xl5.parse("main")
dfH1.to_pickle('rawHouse1.pkl') 
dfH2 = xl6.parse("main")
dfH2.to_pickle('rawHouse2.pkl')
#dfH1.head(5)

In [10]:
### From rawHouse1.pkl clean

temp5 = pd.read_pickle('rawHouse1.pkl')
temp5 = temp5.iloc[2:48]
# temp1 = temp1.replace(['County','\.','\s'],'', regex = True)
cTemp5 = temp5['table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)']
temp5 = temp5.drop(['table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)',
                    'Unnamed: 1','Unnamed: 12','Unnamed: 13'], axis = 1)
temp5 = temp5.astype(int)
temp5.columns = temp5.iloc[0]
temp5 = temp5.iloc[2:]
temp5['County'] = cTemp

house1 = temp5.copy()
house1.head(3)

2,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,County
4,119537,123537,127725,131639,136296,141931,149507,154060,156868,158504,Ada
5,1999,2056,2123,2198,2263,2361,2451,2530,2584,2620,Adams
6,29177,29491,29814,30184,30609,31184,31837,32508,32835,33065,Bannock


In [11]:
### From rawHouse2.pkl clean

temp6 = pd.read_pickle('rawHouse2.pkl')
temp6 = temp6.replace(['Housing Unit Estimate \(as of July 1\) \- ','County, Idaho'],'', regex = True)
temp6.columns = temp6.iloc[0]

cT6 = temp6['Geography']
temp6 = temp6.drop(['Id','Id2','Geography','April 1, 2010 - Census',
                     'April 1, 2010 - Estimates Base'], axis = 1)
temp6 = temp6.astype(int)
temp6['County'] = cT6
temp6 = temp6.iloc[1:]
temp6 = temp6.replace('\s','', regex = True)

house2 = temp6.copy()
house2.head(3)

,2010,2011,2012,2013,2014,2015,2016,2017,2018,County
1,159814,161031,162421,164963,168141,171977,176086,180657,186481,Ada
2,2641,2654,2661,2666,2667,2671,2675,2684,2695,Adams
3,33228,33358,33424,33550,33703,33855,34037,34231,34450,Bannock


### Combine cleaned dataframes to merge into totalHouse and pickle

In [12]:
totalHouse = pd.merge(house2,house1, on = 'County')

# Reorder columns, not all column titles are str some are int, going to 
# transpose df so will turn them all into factors then
totalHouse = totalHouse[["County",2000,2001,2002,2003,2004,2005,2006,2007,
                    2008,2009,"2010","2011","2012","2013","2014","2015",
                    "2016","2017","2018"]]

totalHouse.to_pickle('totalHouse.pkl')
totalHouse.head(3)

,County,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Ada,119537,123537,127725,131639,136296,141931,149507,154060,156868,158504,159814,161031,162421,164963,168141,171977,176086,180657,186481
1,Adams,1999,2056,2123,2198,2263,2361,2451,2530,2584,2620,2641,2654,2661,2666,2667,2671,2675,2684,2695
2,Bannock,29177,29491,29814,30184,30609,31184,31837,32508,32835,33065,33228,33358,33424,33550,33703,33855,34037,34231,34450


### From totalHouse.pkl make metroHouse and pickle it

In [13]:
metroHouseT = pd.read_pickle('totalHouse.pkl')
metroHouseT = metroHouseT.iloc[[0,7,13,22,36],:]

tempH = metroHouseT.copy()
tempH = tempH.T
tempH.columns = tempH.iloc[0]
tempH = tempH.iloc[1:]
tempH = tempH.reset_index()
tempH = tempH.rename(columns={'index':'Year'})

# Add columns together for a total house count
tempH['TotalHouse'] = tempH['Ada']+tempH['Boise']+tempH['Canyon']+tempH['Gem']+tempH['Owyhee']

metroHouse = tempH.copy()
metroHouse = metroHouse.astype('int')

metroHouse['Year'] = metroHouse['Year'].astype(str)
metroHouse['Year'] = metroHouse['Year'].astype('datetime64[Y]')

metroHouse.to_pickle('metroHouse.pkl')
metroHouse.head(3)

County,Year,Ada,Boise,Canyon,Gem,Owyhee,TotalHouse
0,2000-01-01,119537,4361,48483,5919,4464,182764
1,2001-01-01,123537,4464,50560,6055,4511,189127
2,2002-01-01,127725,4536,52790,6161,4541,195753


In [ ]:
metroHouse = pd.read_pickle('metroHouse.pkl')
metroHouse.dtypes

## Create yearTotals from metroHouse.pkl, metroPop.pkl, and job

In [ ]:
metroHouse = pd.read_pickle('metroHouse.pkl')
metroPop = pd.read_pickle('metroPop.pkl')
metroJobs = pd.read_pickle('jobTypes.pkl')


yearTotals = metroHouse.copy()
yearTotals = yearTotals.drop(['Ada','Boise','Gem','Canyon','Owyhee'],axis =1)
yearTotals['TotalPop'] = metroPop['TotalPop']
yearTotals['TotalEmp'] = metroJobs['All Occupations']
yearTotals.to_pickle('yearTotals.pkl')
yearTotals